## Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
import joblib
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.svm import SVC
from xgboost import XGBClassifier
from tensorflow.keras.models import Model, save_model, load_model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Create models directory if it doesn't exist
Path("../models/supervised").mkdir(parents=True, exist_ok=True)
Path("../models/unsupervised").mkdir(parents=True, exist_ok=True)
Path("../models/preprocessing").mkdir(parents=True, exist_ok=True)


## Data Loading and Initial Exploration

In [ ]:
# Data validation utilities
def validate_dataframe(df, name="DataFrame"):
    """Validate DataFrame quality and report statistics."""
    print(f"\n{'='*50}")
    print(f"Validation: {name}")
    print(f"{'='*50}")
    print(f"Shape: {df.shape[0]} rows × {df.shape[1]} columns")
    print(f"Missing values: {df.isnull().sum().sum()}")
    print(f"Duplicate rows: {df.duplicated().sum()}")
    print(f"Data types: {df.dtypes.value_counts().to_dict()}")
    
    if df.select_dtypes(include=[np.number]).shape[1] > 0:
        numeric_df = df.select_dtypes(include=[np.number])
        print(f"Numeric columns: {numeric_df.shape[1]}")
        print(f"  - Mean range: [{numeric_df.mean().min():.2e}, {numeric_df.mean().max():.2e}]")
        print(f"  - Std range: [{numeric_df.std().min():.2e}, {numeric_df.std().max():.2e}]")
    
    print(f"{'='*50}\n")
    return df

def check_class_balance(y, label_name="Labels"):
    """Check and report class distribution."""
    counts = y.value_counts()
    print(f"\n{label_name} distribution:")
    for label, count in counts.items():
        percentage = (count / len(y)) * 100
        print(f"  {label}: {count} ({percentage:.1f}%)")
    
    # Check for severe imbalance
    if len(counts) > 1:
        ratio = counts.max() / counts.min()
        if ratio > 3:
            print(f"⚠ Warning: Class imbalance detected (ratio {ratio:.1f}:1)")
    return counts


In [ ]:
# Use relative path from notebooks/ to data/
file_path = Path("../data/GSE19804_series_matrix.txt")

# Check if file exists
if not file_path.exists():
    print(f"Error: File not found at {file_path}")
    print(f"Please ensure the data file is in the data/ directory")
    sys.exit(1)

# Lire les lignes utiles
try:
    with open(file_path, 'rt') as f:
        lines = f.readlines()
except Exception as e:
    print(f"Error reading file: {e}")
    sys.exit(1)

# Identify start and end lines of the data matrix
start = None
end = None
for i, line in enumerate(lines):
    if line.startswith("!series_matrix_table_begin"):  # start of our matrix
        start = i + 1  # ignore metadata line
    elif line.startswith("!series_matrix_table_end"):  # end of our matrix
        end = i
        break

if start is None or end is None:
    print("Error: Could not find data matrix markers in file")
    sys.exit(1)

# read the data matrix
df = pd.read_csv(file_path, sep="\t", skiprows=start, nrows=end - start - 1, index_col=0, comment="!")
print(f"✓ Data loaded successfully: {df.shape[0]} genes, {df.shape[1]} samples")
df.head()


In [ ]:
print(f"Shape: {df.shape}")
print(f"Samples: {df.columns[:5]}")

# Remove columns with missing names
df = df.loc[:, df.columns.notna()]
print(f"✓ Removed columns with missing names: {df.shape}")

# Remove duplicate gene rows
df = df[~df.index.duplicated(keep='first')]
print(f"✓ Removed duplicate genes: {df.shape}")

# Convert all values to numeric
df = df.apply(pd.to_numeric, errors='coerce')

# Check for missing values
missing_count = df.isnull().sum().sum()
print(f"⚠ Missing values found: {missing_count}")

if missing_count > 0:
    # Count missing per row and column
    missing_per_row = df.isnull().sum(axis=1)
    missing_per_col = df.isnull().sum(axis=0)
    print(f"  - Genes with missing values: {(missing_per_row > 0).sum()}")
    print(f"  - Samples with missing values: {(missing_per_col > 0).sum()}")


In [ ]:
# Strategy: Remove genes (rows) with any missing values
# Keep all samples (columns) since they're our observations
df_clean = df.dropna(axis=0)  # axis=0 removes rows (genes) with NaN
print(f"✓ Removed genes with missing values: {df.shape[0]} → {df_clean.shape[0]} genes")

# Remove genes (columns after transpose) with all zeros (no expression)
df_clean = df_clean.loc[:, (df_clean != 0).any(axis=0)]
print(f"✓ Removed zero-expression genes: {df_clean.shape}")

# Transpose: samples as rows, genes as columns
df_T = df_clean.T
print(f"✓ Transposed data: {df_T.shape[0]} samples × {df_T.shape[1]} genes")

# Standardize features (Z-score normalization)
scaler = StandardScaler()
df_scaled = pd.DataFrame(
    scaler.fit_transform(df_T), 
    index=df_T.index, 
    columns=df_T.columns
)

# Verify cleaning
print(f"\n✓ Final cleaned data: {df_scaled.shape}")
print(f"✓ Missing values: {df_scaled.isnull().sum().sum()}")
print(f"✓ Mean: {df_scaled.mean().mean():.2e} (should be ~0)")
print(f"✓ Std: {df_scaled.std().mean():.2f} (should be ~1)")

# Save scaler and feature names for later use
scaler_path = Path("../models/preprocessing/scaler.pkl")
features_path = Path("../models/preprocessing/feature_names.pkl")

joblib.dump(scaler, scaler_path)
joblib.dump(list(df_scaled.columns), features_path)
print(f"\n✓ Saved scaler to {scaler_path}")
print(f"✓ Saved feature names to {features_path}")


In [ ]:
# Reduced to 2 dimensions for visualization
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(df_scaled)

# Explained variance
explained_var = pca.explained_variance_ratio_
print(f"Variance explained by PC1: {explained_var[0]:.2%}")
print(f"Variance explained by PC2: {explained_var[1]:.2%}")
print(f"Total variance explained: {explained_var.sum():.2%}")

# Store the projection with samples index
df_pca = pd.DataFrame(X_pca, columns=['PC1', 'PC2'], index=df_scaled.index)
print(f"\n✓ PCA projection: {df_pca.shape}")

# Save PCA transformer
pca_path = Path("../models/preprocessing/pca_transformer.pkl")
joblib.dump(pca, pca_path)
print(f"✓ Saved PCA transformer to {pca_path}")


## Anomaly Detection with Isolation Forest

In [ ]:
# Initialize Isolation Forest model
iso_forest = IsolationForest(n_estimators=100, contamination=0.1, random_state=42)

# Train the model on standardized data
iso_forest.fit(df_scaled)

# Predict anomalies: -1 = anomaly, 1 = normal
df_pca['Anomaly_IF'] = iso_forest.predict(df_scaled)

# Save model
if_path = Path("../models/unsupervised/isolation_forest.pkl")
joblib.dump(iso_forest, if_path)
print(f"✓ Saved Isolation Forest model to {if_path}")


In [ ]:
# Create a new text column for display
df_pca['Anomaly_Label'] = df_pca['Anomaly_IF'].map({1: "Normal", -1: "Anomaly"})

# Use this column for color
plt.figure(figsize=(10, 6))
sns.scatterplot(
    data=df_pca,  # DataFrame with principal components and anomaly labels
    x='PC1', y='PC2',  # Principal components for axes
    hue='Anomaly_Label',  # Color based on anomaly label
    palette={"Normal": "blue", "Anomaly": "red"},  # Color palette for labels
    s=80  # Point size
)

plt.title("Anomaly Detection by Isolation Forest")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.legend(title="Predicted Status")
plt.grid(True)
plt.tight_layout()
plt.show()


## Anomaly Detection with Autoencoder

In [ ]:
# Build autoencoder architecture
n_features = df_scaled.shape[1]

input_layer = Input(shape=(n_features,))
encoded = Dense(64, activation='relu')(input_layer)
encoded = Dense(32, activation='relu')(encoded)
decoded = Dense(64, activation='relu')(encoded)
output_layer = Dense(n_features, activation='linear')(decoded)

autoencoder = Model(inputs=input_layer, outputs=output_layer)
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

print(f"Autoencoder architecture: {n_features} → 64 → 32 → 64 → {n_features}")

# Train autoencoder
history = autoencoder.fit(
    df_scaled, df_scaled,
    epochs=50,
    batch_size=16,
    shuffle=True,
    validation_split=0.1,
    verbose=0
)

print(f"✓ Autoencoder training completed")
print(f"  Final loss: {history.history['loss'][-1]:.4f}")
print(f"  Final val_loss: {history.history['val_loss'][-1]:.4f}")

# Save autoencoder model
ae_path = Path("../models/unsupervised/autoencoder.h5")
autoencoder.save(ae_path)
print(f"✓ Saved Autoencoder model to {ae_path}")


In [ ]:
# Reconstruction
reconstructed = autoencoder.predict(df_scaled)
# Mean squared error per sample
reconstruction_error = np.mean(np.square(df_scaled - reconstructed), axis=1)
# Add to PCA DataFrame
df_pca['Reconstruction_Error'] = reconstruction_error


In [ ]:
# Reconstruct df_pca with aligned index
df_pca = pd.DataFrame(X_pca, columns=["PC1", "PC2"], index=df_scaled.index)

# Add reconstruction error
df_pca['Reconstruction_Error'] = reconstruction_error

# Add Isolation Forest predictions
df_pca['Anomaly_IF'] = iso_forest.predict(df_scaled).astype(int)

# AE detection: threshold based on 90th percentile
threshold = np.percentile(reconstruction_error, 90)

# Anomaly detection: -1 = anomaly, 1 = normal
df_pca['Anomaly_AE'] = (df_pca['Reconstruction_Error'] > threshold).astype(int)
df_pca['Anomaly_AE'] = df_pca['Anomaly_AE'].map({0: 1, 1: -1})  # align with Isolation Forest

# Add readable label
df_pca['AE_Label'] = df_pca['Anomaly_AE'].map({1: "Normal", -1: "Anomaly"})


In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(
    data=df_pca,
    x='PC1', y='PC2',
    hue='AE_Label',
    palette={"Normal": "blue", "Anomaly": "red"},
    s=80
)

plt.title("Anomaly Detection by Autoencoder")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.legend(title="Predicted Status")
plt.grid(True)
plt.tight_layout()
plt.show()


## Anomaly Detection with Local Outlier Factor (LOF)

In [ ]:
# Initialize LOF model
lof = LocalOutlierFactor(
    n_neighbors=20,
    contamination=0.1,
    novelty=False  # fit_predict mode only
)

# Predict anomalies directly (LOF doesn't have separate fit/predict)
lof_preds = lof.fit_predict(df_scaled)  # -1 = anomaly, 1 = normal

# Add to PCA dataframe
df_pca['Anomaly_LOF'] = lof_preds 
df_pca['LOF_Label'] = df_pca['Anomaly_LOF'].map({1: "Normal", -1: "Anomaly"})

print(f"✓ LOF predictions completed")

# Note: LOF with novelty=False cannot be saved for prediction on new data
# For deployment, use novelty=True and retrain
print("⚠ LOF model not saved (novelty=False mode cannot predict on new data)")


In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(
    data=df_pca,
    x='PC1', y='PC2',
    hue='LOF_Label',
    palette={"Normal": "blue", "Anomaly": "red"},
    s=80
)

plt.title("Anomaly Detection by Local Outlier Factor")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.legend(title="Predicted Status")
plt.grid(True)
plt.tight_layout()
plt.show()


## Model Comparison Against Ground Truth

In [ ]:
# Extract sample metadata from original file
sample_ids = []
sample_labels = []

# Extract sample IDs
for line in lines:
    if line.startswith("!Sample_geo_accession"):
        sample_ids = [x.replace('"', '').strip() for x in line.strip().split("\t")[1:]]
        break

if not sample_ids:
    print("⚠ Warning: No sample IDs found in file")

# Extract sample labels from titles
for line in lines:
    if line.startswith("!Sample_title"):
        titles = line.strip().split("\t")[1:]
        for title in titles:
            t = title.lower()
            if "normal" in t:
                sample_labels.append("Healthy")
            elif "cancer" in t or "tumor" in t:
                sample_labels.append("Cancer")
            else:
                sample_labels.append("Unknown")
        break

if not sample_labels:
    print("⚠ Warning: No sample labels found in file")
elif len(sample_ids) != len(sample_labels):
    print(f"⚠ Warning: Mismatch between IDs ({len(sample_ids)}) and labels ({len(sample_labels)})")

# Create mapping dictionary
sample_status = dict(zip(sample_ids, sample_labels))

# Map labels to dataframe
df_pca['Label'] = df_pca.index.map(sample_status)

# Check for unmapped samples
unmapped = df_pca['Label'].isna().sum()
if unmapped > 0:
    print(f"⚠ Warning: {unmapped} samples could not be mapped to labels")

# Display distribution
print("\n✓ Sample label distribution:")
print(df_pca['Label'].value_counts(dropna=False))


In [ ]:
def evaluate_model(anomaly_col, model_name):
    """Evaluate unsupervised anomaly detection model against true labels."""
    print(f"\n{'='*60}")
    print(f"Evaluation: {model_name}")
    print(f"{'='*60}")
    
    # Prepare evaluation dataframe
    df_eval = df_pca[['Label', anomaly_col]].copy()
    
    # Clean and standardize labels
    df_eval['Label_clean'] = df_eval['Label'].astype(str).str.lower().str.strip()
    df_eval['Label_clean'] = df_eval['Label_clean'].replace({
        'cancer': 'Cancer',
        'normal': 'Healthy', 
        'tumor': 'Cancer',
        'healthy': 'Healthy'
    })
    
    # Filter valid data only
    data = df_eval[df_eval['Label_clean'].isin(['Cancer', 'Healthy'])].copy()
    
    if data.empty:
        print("⚠ No usable data for evaluation")
        return None
    
    print(f"Evaluating on {len(data)} samples")
    
    # Convert to binary: 1 = Cancer, 0 = Healthy
    y_true = data['Label_clean'].map({'Cancer': 1, 'Healthy': 0})
    y_pred = data[anomaly_col].map({-1: 1, 1: 0})  # -1 (anomaly) = Cancer
    
    # Check for conversion issues
    if y_pred.isna().any():
        print(f"⚠ Warning: {y_pred.isna().sum()} predictions could not be converted")
    
    try:
        # Confusion matrix
        cm = confusion_matrix(y_true, y_pred)
        print("\nConfusion Matrix:")
        print(cm)
        
        # Classification report
        print("\nClassification Report:")
        print(classification_report(y_true, y_pred, target_names=["Healthy", "Cancer"], zero_division=0))
        
        return {'confusion_matrix': cm, 'y_true': y_true, 'y_pred': y_pred}
        
    except Exception as e:
        print(f"✗ Error during evaluation: {e}")
        return None

# Note: Call this function for each model
# Example: evaluate_model('Anomaly_IF', 'Isolation Forest')


In [ ]:
evaluate_model('Anomaly_IF', 'Isolation Forest')  # Evaluate Isolation Forest model


In [ ]:
evaluate_model('Anomaly_AE', 'Autoencoder')  # Evaluate Autoencoder model


In [ ]:
evaluate_model('Anomaly_LOF', 'Local Outlier Factor')  # Evaluate Local Outlier Factor model


| Model                | Accuracy | Recall Cancer | Precision Cancer | F1-score Cancer |
| -------------------- | -------- | ------------- | ---------------- | --------------- |
| **Isolation Forest** | 57%      | 17%           | 83%              | 28%             |
| **Autoencoder**      | 42%      | **2%**        | **8%**           | **3%**          |
| **LOF**              | **53%**  | **13%**       | 67%              | **22%**         |


## Supervised Learning Pipeline

### Data Preparation for Classification

In [ ]:
# Prepare data for supervised learning
X = df_scaled.copy()

# Extract and clean labels
y = df_pca['Label'].astype(str).str.lower().str.strip()
y = y.replace({'cancer': 1, 'healthy': 0, 'normal': 0})

# Remove samples with unknown labels
valid_mask = y.isin([0, 1])
X = X[valid_mask]
y = y[valid_mask].astype(int)

print(f"✓ Prepared supervised learning data:")
print(f"  - Features: {X.shape}")
print(f"  - Labels: {len(y)}")

# Check class balance
check_class_balance(y, "Class distribution")


In [ ]:
# Train/test split with stratification
try:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=0.2, 
        random_state=42, 
        stratify=y
    )
    
    print("✓ Train/test split successful:")
    print(f"  - Training set: {X_train.shape[0]} samples")
    print(f"  - Test set: {X_test.shape[0]} samples")
    
    print("\n  Training set distribution:")
    check_class_balance(y_train, "Train")
    
    print("\n  Test set distribution:")
    check_class_balance(y_test, "Test")
    
except ValueError as e:
    print(f"✗ Error during train/test split: {e}")
    print("This may happen if one class has too few samples for stratification")


In [ ]:
# Train SVM model
print("Training SVM (Linear Kernel)...")

try:
    svm_model = SVC(kernel='linear', random_state=42)
    svm_model.fit(X_train, y_train)
    
    # Predictions
    y_pred_svm = svm_model.predict(X_test)
    
    print("✓ SVM training completed")
    
    # Save model
    svm_path = Path("../models/supervised/svm_model.pkl")
    joblib.dump(svm_model, svm_path)
    print(f"✓ Saved SVM model to {svm_path}\n")
    
    # Evaluation
    print("="*60)
    print("SVM Evaluation Results")
    print("="*60)
    
    cm = confusion_matrix(y_test, y_pred_svm)
    print("\nConfusion Matrix:")
    print(cm)
    
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred_svm, target_names=["Healthy", "Cancer"], zero_division=0))
    
    accuracy = accuracy_score(y_test, y_pred_svm)
    print(f"\nTest Accuracy: {accuracy:.2%}")
    
except Exception as e:
    print(f"✗ Error training SVM: {e}")
    import traceback
    traceback.print_exc()


### SVM Model Training and Evaluation

#### Comparison with Unsupervised Models

| Metric           | Supervised SVM | IF (best unsupervised) |
| ---------------- | -------------- | ---------------------- |
| Recall Cancer    | **0.83**       | 0.17                   |
| Precision Cancer | 0.91           | **0.83**               |
| F1-score Cancer  | **0.87**       | 0.28                   |
| Accuracy         | **0.88**       | 0.57                   |

### XGBoost Model Training and Evaluation

In [ ]:
# Train XGBoost model
print("Training XGBoost Classifier...")

try:
    xgb_model = XGBClassifier(
        eval_metric='logloss',
        random_state=42,
        use_label_encoder=False
    )
    
    xgb_model.fit(X_train, y_train)
    
    # Predictions
    y_pred_xgb = xgb_model.predict(X_test)
    
    print("✓ XGBoost training completed")
    
    # Save model
    xgb_path = Path("../models/supervised/xgboost_model.pkl")
    joblib.dump(xgb_model, xgb_path)
    print(f"✓ Saved XGBoost model to {xgb_path}\n")
    
    # Evaluation
    print("="*60)
    print("XGBoost Evaluation Results")
    print("="*60)
    
    cm = confusion_matrix(y_test, y_pred_xgb)
    print("\nConfusion Matrix:")
    print(cm)
    
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred_xgb, target_names=["Healthy", "Cancer"], zero_division=0))
    
    accuracy = accuracy_score(y_test, y_pred_xgb)
    print(f"\nTest Accuracy: {accuracy:.2%}")
    
except Exception as e:
    print(f"✗ Error training XGBoost: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
# Extract feature importances
importances = xgb_model.feature_importances_
feature_names = X.columns  # gene names

# Create sorted DataFrame
feat_imp = pd.DataFrame({
    'Gene': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False).head(20)

plt.figure(figsize=(10, 6))
sns.barplot(data=feat_imp, x='Importance', y='Gene', palette='viridis')
plt.title("Top 20 Most Discriminative Genes According to XGBoost")
plt.xlabel("Importance Score")
plt.ylabel("Gene")
plt.grid(True, linestyle='--', alpha=0.3)
plt.tight_layout()
plt.show()


### Model Predictions Visualization in PCA Space

In [ ]:
# Add predictions to df_pca (on test set only)
df_pca.loc[X_test.index, 'Pred_SVM'] = y_pred_svm  # values 0/1
df_pca.loc[X_test.index, 'Pred_XGB'] = y_pred_xgb 
# Add columns with readable labels for SVM and XGBoost
df_pca['SVM_Label'] = df_pca['Pred_SVM'].map({1: 'Cancer', 0: 'Healthy'}) 
df_pca['XGB_Label'] = df_pca['Pred_XGB'].map({1: 'Cancer', 0: 'Healthy'})


In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(
    data=df_pca.loc[X_test.index],
    x='PC1', y='PC2',
    hue='SVM_Label',
    palette={'Healthy': 'blue', 'Cancer': 'red'},
    s=80
)
plt.title("SVM Model Predictions (on test data)")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.legend(title="Predicted Class")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(
    data=df_pca.loc[X_test.index],
    x='PC1', y='PC2',
    hue='XGB_Label',
    palette={'Healthy': 'blue', 'Cancer': 'red'},
    s=80
)
plt.title("XGBoost Model Predictions (on test data)")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.legend(title="Predicted Class")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
top_genes = feat_imp['Gene'].tolist()[:20]  # top 20 genes

# Extract their columns from df_scaled
X_subset = df_scaled[top_genes]

# Calculate correlation matrix
corr_matrix = X_subset.corr()

# Display with seaborn
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', square=True, cbar_kws={"shrink": .6})
plt.title("Correlation Matrix of the 20 Most Discriminative Genes (XGBoost)")
plt.tight_layout()
plt.show()
